In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
# loading the directories 
# importing the libraries
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.applications import VGG16
#from keras.preprocessing import image
num_classes=3
IMAGE_SHAPE = [224, 224]  # we will keep the image size as (64,64). You can increase the size for better results. 
batch_size=32
# loading the weights of VGG16 without the top layer. These weights are trained on Imagenet dataset.
vgg = VGG16(input_shape = (224,224,3), weights = 'imagenet', include_top = False)  # input_shape = (64,64,3) as required by VGG

# this will exclude the initial layers from training phase as there are already been trained.
for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
x = Dense(128, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
x = Dense(64, activation = 'relu')(x) 
x = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

model = Model(inputs = vgg.input, outputs = x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [3]:
from keras.preprocessing.image import ImageDataGenerator
trdata = ImageDataGenerator()
train_data_gen = trdata.flow_from_directory(directory="Train",target_size=(224,224),shuffle=False, class_mode='categorical')
tsdata = ImageDataGenerator()
test_data_gen = tsdata.flow_from_directory(directory="Test", target_size=(224,224),shuffle=False, class_mode='categorical')


Using TensorFlow backend.


Found 385 images belonging to 3 classes.
Found 138 images belonging to 3 classes.


In [4]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
epochs = 5
#checkpoint = ModelCheckpoint(filepath='finalvgg16model.h5', verbose=1, save_best_only=True)
training_steps_per_epoch = np.ceil(train_data_gen.samples / batch_size)
validation_steps_per_epoch = np.ceil(test_data_gen.samples / batch_size)
    
model.fit_generator(train_data_gen, steps_per_epoch=training_steps_per_epoch, validation_data=test_data_gen, validation_steps=validation_steps_per_epoch,
                        epochs=epochs, verbose=1)
print('Training Completed!')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
13/13 [==============================] - ETA: 0s - loss: 62.0823 - accuracy: 0.4364WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x000001E6442E2558> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, se

In [5]:
Y_pred = model.predict(test_data_gen, test_data_gen.samples / batch_size)
val_preds = np.argmax(Y_pred, axis=1)
import sklearn.metrics as metrics
val_trues =test_data_gen.classes
from sklearn.metrics import classification_report
print(classification_report(val_trues, val_preds))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        59
           1       0.91      0.96      0.94        53
           2       0.78      0.69      0.73        26

    accuracy                           0.88       138
   macro avg       0.86      0.85      0.86       138
weighted avg       0.88      0.88      0.88       138



In [6]:
Y_pred = model.predict(test_data_gen, test_data_gen.samples / batch_size)
val_preds = np.argmax(Y_pred, axis=1)
import sklearn.metrics as metrics
val_trues =test_data_gen.classes
cm = metrics.confusion_matrix(val_trues, val_preds)
cm

array([[53,  3,  3],
       [ 0, 51,  2],
       [ 6,  2, 18]], dtype=int64)

In [10]:
keras_file="Model.h5"
tf.keras.models.save_model(model,keras_file)

In [15]:
#Test the model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
img_path = 'fresh.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds=model.predict(x)
# create a list containing the class labels
class_labels = ['Apple','Banana','Orange']

# find the index of the class with maximum score
pred = np.argmax(preds, axis=-1)

# print the label of the class with maximum score
print(class_labels[pred[0]])

Orange
